# AI Response Comparison: <br>Simple vs. Elaborated Prompts

### This notebook demonstrates how to generate <br>and compare responses from an AI model using both <br>simple and elaborated prompts. 

# Imports

We import the necessary libraries:

In [4]:
import pandas as pd
import re
import google.generativeai as genai

# Setting up the API key for gemini

In [ ]:
genai.configure(api_key='YOUR_API_KEY')
model = genai.GenerativeModel('gemini-pro')

# Simple user prompt 

In [13]:
# User input
user_request = "I would like to learn running. Give me recommendations for this hobby with detailed analysis."
simple_response = model.generate_content(user_request).text         

In [14]:
print(simple_response)

**Getting Started with Running**

**1. Set Realistic Goals:**
* Start with a manageable distance and duration, such as 1-mile runs for 15 minutes.
* Gradually increase distance and intensity as you progress.

**2. Proper Gear:**
* Choose comfortable running shoes with good support.
* Wear moisture-wicking clothing to stay dry and prevent chafing.
* Consider a GPS watch or fitness tracker to monitor distance and pace.

**3. Find a Training Program:**
* Join a running group or consult with a personal trainer for guidance.
* Follow a structured training plan that gradually increases workload and reduces risk of injury.

**4. Listen to Your Body:**
* Rest when needed.
* Take breaks if you experience pain or discomfort.
* Gradually increase distance and intensity to avoid overexertion.

**Benefits of Running**

**Physical Benefits:**
* Improves cardiovascular health
* Strengthens muscles and bones
* Reduces body fat and promotes weight loss
* Boosts metabolism

**Mental Benefits:**
* Reduce

# Elaborated prompt

In [ ]:
def create_elaborate_prompt(user_request):
    return f"""You are an expert hobby advisor specializing in fitness and running. Your task is to create a detailed, data-driven 
analysis and recommendation plan for someone interested in taking up running as a hobby.
Here's the user request: {user_request}
Please structure your response in the following way:

1. Initial Assessment
- Analyze why running is a good hobby choice
- List the primary benefits (physical, mental, social)
- Identify potential challenges for beginners

2. Getting Started Plan
- Required equipment and estimated costs
- Recommended schedule for beginners
- Safety considerations and prerequisites

3. Progression Timeline
- Week-by-week breakdown for the first 8 weeks
- Monthly milestones for the first 6 months
- Long-term goals and achievements

4. Data Analysis
Please include data tables in your analysis with the following structure:

Equipment Costs:
| Item | Essential | Average Cost | Lifetime (Months) |
|------|-----------|--------------|-------------------|
Weekly Progress:
| Week | Running Minutes | Walking Minutes | Total Distance (Km) | Sessions Per Week |
|------|----------------|-----------------|-------------------|------------------|
Monthly Milestones:
| Month | Distance Goal (Km) | Avg Pace (Min/Km) | Long Run (Km) |
|-------|-------------------|-------------------|----------------|
5. Visualization Suggestions
- Weekly distance progression
- Pace improvements
- Heart rate zones
- Recovery metrics
6. Community and Support
- Recommended apps and tools
- Local groups or communities
- Online resources
7. Risk Analysis
Include a risk assessment matrix for common challenges and solutions.
Please provide a comprehensive response in a clear, structured manner with specific, actionable recommendations.
"""

#### Extract Tables from Text

Explanation :

The extract_tables function parses markdown tables from the AI's text response and converts them into Pandas DataFrames.

In [9]:
def extract_tables(text):
    """Extract markdown tables from text"""
    table_pattern = r"\|.*?\|[\r\n][-|:\s]+[\r\n](?:\|.*?\|[\r\n])+"
    tables = []
    
    for table_match in re.finditer(table_pattern, text, re.MULTILINE):
        table_text = table_match.group(0)
        
        lines = [line.strip() for line in table_text.split('\n') if line.strip()]
        headers = [col.strip() for col in lines[0].split('|')[1:-1]]
        data = []
        for line in lines[2:]:
            row = [cell.strip() for cell in line.split('|')[1:-1]]
            data.append(row)
        df = pd.DataFrame(data, columns=headers)
        tables.append(df)
        
    return tables

#### Extract Sections from Text

Explanation

The extract_sections function identifies and extracts different sections from the AI's response based on headers or numbered points.

In [10]:
def extract_sections(text):
    """Extract sections based on numbered points or headers"""
    sections = {}
    current_section = None
    current_content = []
    
    for line in text.split('\n'):
        # Check for section headers (numbered points or markdown headers)
        if re.match(r'^\d+\.|\#\#?|^\*\*', line.strip()):
            if current_section and current_content:  # Only save if there's content
                content = '\n'.join(current_content).strip()
                if content:  # Only save if content is not empty
                    sections[current_section] = content
            current_section = line.strip('*# \n')
            current_content = []
        else:
            if current_section:
                current_content.append(line)
    
    # Add the last section if it has content
    if current_section and current_content:
        content = '\n'.join(current_content).strip()
        if content:
            sections[current_section] = content
    
    return sections

### Test

In [ ]:
from tabulate import tabulate
elaborate_prompt = create_elaborate_prompt(user_request)
elaborate_response = model.generate_content(elaborate_prompt).text

# Extract and display sections
sections = extract_sections(elaborate_response)
if sections:  # Only show sections header if there are sections

    for section_title, content in sections.items():
        # Only create expander if content exists and is not just whitespace
        if content and not content.isspace():
            print(section_title)
            print("************************************")
            print(content)

# Extract and display tables
elaborate_tables = extract_tables(elaborate_response)
if elaborate_tables:
    print("Tables Found in Elaborated Response")
    for i, table in enumerate(elaborate_tables):
        if not table.empty:  # Only show non-empty tables
            print(tabulate(table, headers='keys', tablefmt='psql'))
            




Why Running is a Good Hobby Choice:
************************************
* **Accessible and Convenient:** Running requires minimal equipment and can be done almost anywhere, making it a highly accessible activity.
* **Improves Physical Health:** Running is a cardiovascular exercise that strengthens the heart, lungs, and muscles.
* **Enhances Mental Well-being:** Exercise releases endorphins, which have mood-boosting effects. Running can help reduce stress, anxiety, and depression.
* **Promotes Social Connections:** Joining running groups or participating in races provides opportunities for socialization and community building.
Physical:
************************************
* Improved cardiovascular health
* Increased muscle strength and endurance
* Reduced risk of chronic diseases (e.g., heart disease, type 2 diabetes)
* Weight management
Mental:
************************************
* Reduced stress and anxiety
* Improved mood and sleep quality
* Increased self-esteem and confidence
So